In [ ]:
# all data is output as a data frame -- basically a table you can manipulate
# dfs have sql functions but with their own syntax
# resources: 
# https://codeburst.io/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e
# https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

In [11]:
import pyEX as p # iEX finance unofficial library
import pandas as pd
import csv
import os
import datetime
from pathlib import Path

In [2]:
 DATA_DIR = "Data"

In [3]:
def loadData():
    STOCKS_DIR = os.path.join(DATA_DIR, "stockList.csv")
    
    topCompanies = list(csv.DictReader(open(STOCKS_DIR)))

    companies = []
    for company in topCompanies:
        companies.append(company['name'])

    # initialize dictionary with companies
    data = {company:{} for company in companies}

    for company in topCompanies:
        data[company['name']]['symbol']=company['symbol']
        data[company['name']]['industry_id']=company['industry_id']
        
    return data

In [30]:
def createStockDf(companies_df):
    # arbitrarily initialize df object
    arbSymbol = 'AAPL'
    # get stock history for last month
    arbChart = p.chartDF(arbSymbol, timeframe='1m')
    # add stock symbol to df to act as a foreign key for company table
    arbChart['symbol']='Arbitrary'
    arbChart['industry_id']=0
    # get company info
    arbCompany = p.companyDF(arbSymbol)
    # join the 2 tables and add to rest of stocks df
    stocks_df = pd.merge(arbChart,arbCompany,how='left',on=['symbol'])

    for company in companies_df.values():
        symbol = company['symbol']
        industry_id = company['industry_id']
        # get stock history for last 1 month
        chart_df = p.chartDF(symbol, timeframe='1y')
        # add stock symbol to df to act as a foreign key for company table
        chart_df['symbol']=symbol
        # add our unique identifier for industry to the table
        chart_df['industry_id']=industry_id
        # get company info
        company_df = p.companyDF(symbol)
        # join the 2 tables and add to rest of stocks df
        stocks_df = stocks_df.append(pd.merge(chart_df,company_df,how='left',on=['symbol']))
    
    # delete arbitrary rows
    stocks_df = stocks_df.drop(stocks_df[stocks_df.symbol=='Arbitrary'].index)
    
    return stocks_df

In [12]:
def outputStockData():
    
    data = loadData()

    stocks_df = createStockDf(data)
    
    # File path for this file
    file_name = datetime.datetime.today().strftime('%Y-%m-%d') + '_stockData' + '.csv'
    thispath = Path().absolute()
    OUTPUT_DIR = os.path.join(DATA_DIR, file_name)
    # if the following line throws an error, use the line after to save in same folder
    pd.DataFrame.to_csv(stocks_df, path_or_buf=OUTPUT_DIR, index=False, encoding='utf-8')
    #pd.DataFrame.to_csv(X, path_or_buf=file_name)
    
def main(): # Stuff to do when run from the command line    
    outputStockData()
    pass 

In [13]:
outputStockData()